In [41]:
import os
import math
import random
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2

from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [37]:
BASE_DIR = "datasets/"
names = ["Mask", "No Mask"]

tf.random.set_seed(1)

# Load Model

In [38]:
model_faceDetection = load_model("Model/model_FeceDetection_ResNet50.h5")
print('Model Loaded')

Model Loaded


In [39]:
model_faceDetection.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 23,536,641
Non-trainable params: 53,120
_________________________________________________________________


# Face Detection With OpenCV

In [53]:
def detect_face_mask(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model_faceDetection.predict_classes(images, batch_size=10)
        
    return classes[0][0]

In [63]:
def detect_face(img):
    haar = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    face_detection = haar.detectMultiScale(img)
    
    return face_detection

In [64]:
def draw_ped(img,label,x0,y0,xt,yt,color=(0,0,255),text_color=(255,255,255)):
    (w,h), baseline = cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,1)
    
    cv2.rectangle(img,
                 (x0, y0 + baseline),
                 (max(xt, x0+ w), yt),
                 color,
                 2)
    cv2.rectangle(img,
                 (x0, y0 - h),
                 (x0 + w, y0 + baseline),
                 color,
                 -1)
    cv2.putText(img,
               label,
               (x0,y0),
               cv2.FONT_HERSHEY_COMPLEX,
               0.5,
               text_color,
               1,
               cv2.LINE_AA)
    return img

In [65]:
cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    
    # call the detection method
    img = cv2.resize(frame, (224,224))
    
    y_pred = detect_face_mask(img)
    
    face_detection = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for (x,y,w,h) in face_detection:
        
        if y_pred == 0:
            draw_ped(frame,names[0],x,y,x+w,y+h, color=(0,255,0))
        else:
            draw_ped(frame,names[1],x,y,x+w,y+h, color=(0,0,255))
    
    
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) == ord('q'):
        break
        

cv2.destroyAllWindows()        